### Постановка задачи
Получите данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

Заполните отсутствующие значения по погоде интерполяционными данными.

Разделите данные на обучающие/проверочные в пропорции 80/20.

Постройте (1) первый набор моделей линейной регрессии по часам для каждого из первых 20 зданий по следующим параметрам: air_temperature, dew_temperature, cloud_coverage, wind_speed, sea_level_pressure.

Постройте для этих же 20 зданий (2) второй набор моделей линейной регрессии по часам по параметрам: дни недели и праздники (is_holiday). Требуется построить еще 480 моделей.

Используйте логарифм целевого показателя (meter_reading_log).

Найдите, какой набор моделей работает точнее для первого здания (building_id = 0): у какого набора будет меньше метрика качества?

Данные: 
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


### Загрузка данных, отсечение 20 зданий, объединение и оптимизация

In [2]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
weather = weather[weather["site_id"] == 0]
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[energy["building_id"]<20]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
del buildings
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   building_id    175680 non-null  int64  
 1   meter          175680 non-null  int64  
 2   timestamp      175680 non-null  object 
 3   meter_reading  175680 non-null  float64
 4   site_id        175680 non-null  int64  
 5   primary_use    175680 non-null  object 
 6   square_feet    175680 non-null  int64  
 7   year_built     175680 non-null  float64
 8   floor_count    0 non-null       float64
dtypes: float64(3), int64(4), object(2)
memory usage: 13.4+ MB
None


### Проведем интерполяция данных

In [4]:
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')

### Объединим погодные данные с основными

In [5]:
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "floor_count",
                              "primary_use", "year_built"], axis=1)
del weather
energy = reduce_mem_usage(energy)
print (energy.info())

Потребление памяти меньше на 9.88 Мб (минус 67.0 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           175680 non-null  datetime64[ns]
 1   building_id         175680 non-null  int8          
 2   meter_reading       175680 non-null  float16       
 3   square_feet         175680 non-null  int32         
 4   air_temperature     175680 non-null  float16       
 5   cloud_coverage      175680 non-null  float16       
 6   dew_temperature     175680 non-null  float16       
 7   precip_depth_1_hr   175660 non-null  float16       
 8   sea_level_pressure  175680 non-null  float16       
 9   wind_direction      170680 non-null  float16       
 10  wind_speed          175680 non-null  float16       
dtypes: datetime64[ns](1), float16(8), int32(1), int8(1)
memory usage: 4.9 MB
None


### Добавим час, день недели и праздники в данные

In [6]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
energy['is_holiday'] = (energy['date'].isin(us_holidays)).astype("int8")
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

### Разделим данные

In [7]:
energy_train, energy_test = train_test_split(energy[energy["meter_reading"]>0], test_size=0.2)
print (energy_train.head())

                 timestamp  building_id  meter_reading  square_feet  \
113241 2016-08-23 22:00:00            1       133.2500         2720   
157751 2016-11-24 15:00:00           11       436.7500        49073   
93691  2016-07-14 04:00:00           11       494.2500        49073   
143965 2016-10-26 22:00:00            5         8.1875         8000   
87507  2016-07-01 07:00:00            7       592.5000       121074   

        air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  \
113241        32.812500             6.0        21.093750                0.0   
157751        23.296875             4.0        17.203125                0.0   
93691         23.906250             6.0        21.703125                0.0   
143965        25.593750             4.0        18.296875               -1.0   
87507         24.406250             6.0        22.796875                0.0   

        sea_level_pressure  wind_direction  ...  is_wday0  is_wday1  is_wday2  \
113241           

### Построим массив моделей линейной регрессии - по зданию и часу, всего 480 моделей

In [8]:
hours = range(0, 24)
buildings = range(0, energy_train["building_id"].max() + 1)
lr_columns = ["meter_reading_log", "hour", "building_id",
             "air_temperature", "dew_temperature",
             "sea_level_pressure", "wind_speed", "cloud_coverage"]
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns)
energy_lr = [[]]*len(buildings)
for building in buildings:
    energy_lr[building] = [[]]*len(hours)
    energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
    for hour in hours:
        energy_train_bh = energy_train_b[energy_train_b["hour"]==hour]
        y = energy_train_bh["meter_reading_log"]
        x = energy_train_bh.drop(labels=["meter_reading_log",
            "hour", "building_id"], axis=1)
        model = LinearRegression(fit_intercept=False).fit(x, y)
        energy_lr[building][hour] = model.coef_
        energy_lr[building][hour] = np.append(energy_lr[building][hour], model.intercept_)
print (energy_lr)

[[array([ 0.02070111,  0.01584092,  0.00459455, -0.00551344, -0.00848863,
        0.        ]), array([ 0.03337953,  0.00639256,  0.00450335, -0.0096515 , -0.0087846 ,
        0.        ]), array([ 0.02721422,  0.01045949,  0.0045669 , -0.00368836, -0.00646735,
        0.        ]), array([ 0.02652469,  0.01164939,  0.00455899, -0.00384086, -0.00511576,
        0.        ]), array([ 0.0371316 ,  0.00351011,  0.00449776, -0.00912784, -0.00649291,
        0.        ]), array([ 0.0514999 , -0.00545551,  0.00438916,  0.00228408, -0.01801597,
        0.        ]), array([ 0.03775295, -0.00329486,  0.00462123, -0.00380303, -0.00429326,
        0.        ]), array([ 0.02831487,  0.00839985,  0.00464344, -0.00037902, -0.01293205,
        0.        ]), array([ 0.04364011, -0.00733018,  0.00461021, -0.00978863, -0.00374515,
        0.        ]), array([ 0.05763966, -0.01792445,  0.00447828, -0.00544286, -0.00388542,
        0.        ]), array([ 0.07975613, -0.037561  ,  0.00443226, -0.00902633,

### Рассчитаем качество построенных моделей

In [9]:
def calculate_model (x):
    model = energy_lr[x.building_id][x.hour]
    lr = np.sum([x[col] * model[i] for i,col in enumerate(lr_columns[3:])])
    lr += model[len(lr_columns)-3]
    lr = np.exp(lr)
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) -
                               np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model,
                                    axis=1, result_type="expand")

### Построим модели по дате

In [10]:
lr_columns = ["meter_reading_log", "hour", "building_id", "is_holiday"]
for wday in range(0,7):
    lr_columns.append("is_wday" + str(wday))
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns)
energy_lr = [[]]*len(buildings)
for building in buildings:
    energy_lr[building] = [[]]*len(hours)
    energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
    for hour in hours:
        energy_train_bh = energy_train_b[energy_train_b["hour"]==hour]
        y = energy_train_bh["meter_reading_log"]
        x = energy_train_bh.drop(labels=["meter_reading_log",
            "hour", "building_id"], axis=1)
        model = LinearRegression(fit_intercept=False).fit(x, y)
        energy_lr[building][hour] = model.coef_
        energy_lr[building][hour] = np.append(energy_lr[building][hour], model.intercept_)

### Рассчитаем качество моделей по дате

In [11]:
def calculate_model_is (x):
    model = energy_lr[x.building_id][x.hour]
    if model[0] != 0:
        lr = np.sum([x[col] * model[i] for i,col in enumerate(lr_columns[3:])])
        lr += model[len(lr_columns)-3]
        lr = np.exp(lr)
    if lr < 0:
        lr = 0
    x["meter_reading_is_q"] = (np.log(x.meter_reading + 1) -
                               np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model_is,
                                    axis=1, result_type="expand")

### Найдем, какой набор моделей оказался лучше для 1 здания (метрика качества меньше)

In [12]:
energy_test_0_sum = energy_test[energy_test["building_id"] == 0].sum()
if energy_test_0_sum["meter_reading_is_q"] > energy_test_0_sum["meter_reading_lr_q"]:
    print ("Лучше линейная регрессия по погоде")
else:
    print ("Лучше линейная регрессия по дате")

Лучше линейная регрессия по погоде


C:\Users\NRKP\AppData\Local\Temp\ipykernel_7896\1224244816.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  energy_test_0_sum = energy_test[energy_test["building_id"] == 0].sum()
